### LLM-Prompt

In [1]:
LLM_PROMPT = """
Por favor ayúdame a resolver el siguiente acertijo y dame la respuesta en formato JSON, como una lista de diccionarios. 

En una calle hay cinco casas, cada una de un color distinto. 
En cada casa vive una persona de distinta nacionalidad. 
Cada dueño bebe un único tipo de bebida,
ve una sola Serie de TV y tiene una mascota diferente a sus vecinos.  

1. El brasilero vive en la casa roja.
2. El venezolano tiene un perro como mascota.
3. El ecuatoriano toma té.
4. La peruana vive en la primera casa.
5. La colombiana ve "Westworld".
6. La casa verde está inmediatamente a la izquierda de la blanca
7. El dueño de la casa verde bebe café.
8. El propietario que ve "The Boys" cría pájaros.
9. El dueño de la casa amarilla ve "Dexter".
10. El hombre que vive en la casa del centro bebe leche.
11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato.
12. El hombre que tiene un caballo vive al lado del que ve "Dexter".
13. El propietario que ve "Breaking Bad" toma cerveza.
14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua.
15. La peruana vive al lado de la casa azul.

Cada diccionario en tu respuesta en formato JSON debe tener exactamente las siguientes llaves y ninguna otra:  
"nacionalidad", "bebida_que_toma", "serie_que_ve", "color", "mascota".
"""

### Condiciones en Español

In [2]:
CONDICIONES_ESPANHOL = [ line.split(".")[1].strip() 
                                 for line in 
                                 """0. En una calle hay cinco casas.
                                    1. El brasilero vive en la casa roja.
                                    2. El venezolano tiene un perro como mascota.
                                    3. El ecuatoriano toma té.
                                    4. La peruana vive en la primera casa.
                                    5. La colombiana ve "Westworld".
                                    6. La casa verde está inmediatamente a la izquierda de la blanca
                                    7. El dueño de la casa verde bebe café.
                                    8. El propietario que ve "The Boys" cría pájaros.
                                    9. El dueño de la casa amarilla ve "Dexter".
                                    10. El hombre que vive en la casa del centro bebe leche.
                                    11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato.
                                    12. El hombre que tiene un caballo vive al lado del que ve "Dexter".
                                    13. El propietario que ve "Breaking Bad" toma cerveza.
                                    14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua.
                                    15. La peruana vive al lado de la casa azul."""
                                     .split("\n")
                               ]
CONDICIONES_ESPANHOL

['En una calle hay cinco casas',
 'El brasilero vive en la casa roja',
 'El venezolano tiene un perro como mascota',
 'El ecuatoriano toma té',
 'La peruana vive en la primera casa',
 'La colombiana ve "Westworld"',
 'La casa verde está inmediatamente a la izquierda de la blanca',
 'El dueño de la casa verde bebe café',
 'El propietario que ve "The Boys" cría pájaros',
 'El dueño de la casa amarilla ve "Dexter"',
 'El hombre que vive en la casa del centro bebe leche',
 'El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato',
 'El hombre que tiene un caballo vive al lado del que ve "Dexter"',
 'El propietario que ve "Breaking Bad" toma cerveza',
 'El vecino que ve "El Patrón del Mal" vive al lado del que toma agua',
 'La peruana vive al lado de la casa azul']

## Modelado y Verificación precisa de condiciones en Python

### Clases Auxiliares

In [ ]:

from typing import TypedDict, NotRequired
from dataclasses import dataclass
import pandas as pd 
import sys

class Casa(TypedDict):
    nacionalidad: NotRequired[str]
    color: NotRequired[str]
    serie_que_ve: NotRequired[str]
    bebida_que_toma: NotRequired[str]
    mascote: NotRequired[str]


@dataclass
class NumCasas:
    n: int

    def verificar(self, posible_solucion: list[Casa]) -> bool:
        return len(posible_solucion) == self.n


def _verificar_una(condicion: Casa, miembro_solucion: Casa) -> bool:
    for condicion_llave, condicion_valor in condicion.items():
        if miembro_solucion[condicion_llave].lower()[:3] != condicion_valor.lower()[:3]:
            return False
    return True


@dataclass
class Miembro:
    condicion: Casa

    def verificar(self, posible_solucion: list[Casa]) -> bool:
        return any(
            _verificar_una(condicion=self.condicion, miembro_solucion=miembro)
            for miembro in posible_solucion
        )

@dataclass
class MiembroEnPosicion:
    condicion: Casa
    posicion: int

    def verificar(self, posible_solucion: list[Casa]) -> bool:
        return _verificar_una(
            miembro_solucion=posible_solucion[self.posicion],
            condicion=self.condicion,
        )

@dataclass
class AlLadoDe:
    cond1: Casa
    cond2: Casa

    def verificar(self, posible_solucion: list[Casa]) -> bool:
        
        cond1_izq_cond2 = [ 
            _verificar_una(condicion=self.cond1, miembro_solucion=posible_solucion[i]) 
            and 
            _verificar_una(condicion=self.cond2, miembro_solucion=posible_solucion[i+1])
            for i in range(0, len(posible_solucion) - 1)
        ]

        cond2_izq_cond1 = [ 
            _verificar_una(condicion=self.cond2, miembro_solucion=posible_solucion[i]) 
            and 
            _verificar_una(condicion=self.cond1, miembro_solucion=posible_solucion[i + 1])
            for i in range(0, len(posible_solucion) - 1)
        ]

        return any(cond1_izq_cond2) or any(cond2_izq_cond1)

        

### Condiciones codificadas 

In [ ]:
CONDICIONES = [
    # 0. En una calle hay 5 casas
    NumCasas(5), 
    # 1. El brasilero vive en la casa roja.
    Miembro({"nacionalidad": "brasilero", "color": "roja"}),
    # 2. El venezolano tiene un perro como mascota.
    Miembro({"nacionalidad": "venezolano", "mascota": "perro"}),
    # 3. El ecuatoriano toma té.
    Miembro({"nacionalidad": "ecuatoriano", "bebida_que_toma": "té"}),
    # 4. La peruana vive en la primera casa.
    MiembroEnPosicion({"nacionalidad": "peruana"}, posicion=0),
    # 5. La colombiana ve "Westworld".
    Miembro({"nacionalidad": "colombiana", "serie_que_ve": "Westworld"}),
    # 6. La casa verde está inmediatamente a la izquierda de la blanca
    AlLadoDe({"color": "verde"}, {"color": "blanca"}),
    # 7. El dueño de la casa verde bebe café.
    Miembro({"color": "verde", "bebida_que_toma": "café"}), 
    # 8. El propietario que ve "The Boys" cría pájaros.
    Miembro({"serie_que_ve": "The Boys", "mascota": "pájaros"}),
    # 9. El dueño de la casa amarilla ve "Dexter".
    Miembro({"color": "amarilla", "serie_que_ve": "Dexter"}),   
    # 10. El hombre que vive en la casa del centro bebe leche.
    MiembroEnPosicion({"bebida_que_toma": "leche"}, posicion=2),
    # 11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato.
    AlLadoDe({"serie_que_ve": "El Patrón del Mal"}, {"mascota": "gato"}),
    # 12. El hombre que tiene un caballo vive al lado del que ve "Dexter".
    AlLadoDe({"mascota": "caballo"}, {"mascota": "gato"}),
    # 13. El propietario que ve "Breaking Bad" toma cerveza.
    Miembro({"bebida_que_toma": "cerveza", "serie_que_ve": "Breaking Bad"}),
    # 14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua.
    AlLadoDe({"serie_que_ve": "El Patrón del Mal"}, {"bebida_que_toma": "agua"}),
    # 15. La peruana vive al lado de la casa azul.
    AlLadoDe({"nacionalidad": "peruana"}, {"color": "azul"})
]        

### Función maestra de verifiación y reporte de fallos

In [26]:
def verificar_condiciones(posible_solucion: list[Casa]):

    rows = []
    
    for i, (condicion, espanhol) in enumerate(zip(CONDICIONES, CONDICIONES_ESPANHOL)):
        ok = condicion.verificar(posible_solucion)
        # print(f"condicion #{i}: {condicion}\n\tRESULTADO: {'OK' if ok else 'FALLO'}\n")    
        rows.append({
          "indice": i,
          # "condicion": condicion,
          "en espanhol": espanhol,
          "se cumple": ok
        })

    ret = pd.DataFrame(rows)
    num_fallos = (~ret["se cumple"]).sum()

    if num_fallos > 0:
        print(f"\nSolución INVÁLIDA: Número total de fallos: {num_fallos}"
                "\nLas siguientes no se cumplen:\n", file=sys.stderr)
        no_se_cumplen = ret["en espanhol"][~ret["se cumple"]]
        for i, condicion in no_se_cumplen.items():
            print(f"{i}. {condicion}", file=sys.stderr)
        print("\nPor favor intenta nuevamente.", file=sys.stderr)
    else:
        print(f"\nSolución VÁLIDA!!!")
   
    return ret

### ChatGPT


In [25]:
posible_solucion_chatgpt_1 = [
  {
    "nacionalidad": "peruana",
    "bebida_que_toma": "agua",
    "serie_que_ve": "Dexter",
    "color": "amarilla",
    "mascota": "gato"
  },
  {
    "nacionalidad": "colombiana",
    "bebida_que_toma": "té",
    "serie_que_ve": "Westworld",
    "color": "azul",
    "mascota": "caballo"
  },
  {
    "nacionalidad": "ecuatoriano",
    "bebida_que_toma": "leche",
    "serie_que_ve": "El Patrón del Mal",
    "color": "roja",
    "mascota": "pez"
  },
  {
    "nacionalidad": "venezolano",
    "bebida_que_toma": "cerveza",
    "serie_que_ve": "Breaking Bad",
    "color": "verde",
    "mascota": "perro"
  },
  {
    "nacionalidad": "brasilero",
    "bebida_que_toma": "café",
    "serie_que_ve": "The Boys",
    "color": "blanca",
    "mascota": "pájaro"
  }
]

pd.DataFrame(posible_solucion_chatgpt_1)

,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,peruana,agua,Dexter,amarilla,gato
1,colombiana,té,Westworld,azul,caballo
2,ecuatoriano,leche,El Patrón del Mal,roja,pez
3,venezolano,cerveza,Breaking Bad,verde,perro
4,brasilero,café,The Boys,blanca,pájaro


In [27]:
verificar_condiciones(posible_solucion_chatgpt_1)


Solución INVÁLIDA: Número total de fallos: 5
Las siguientes no se cumplen:

1. El brasilero vive en la casa roja
3. El ecuatoriano toma té
7. El dueño de la casa verde bebe café
11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato
14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua

Por favor intenta nuevamente.


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,False
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,False
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",True
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,False
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",True


In [30]:
posible_solucion_chatgpt_2 = [
  {
    "nacionalidad": "peruana",
    "bebida_que_toma": "agua",
    "serie_que_ve": "Dexter",
    "color": "amarilla",
    "mascota": "gato"
  },
  {
    "nacionalidad": "colombiana",
    "bebida_que_toma": "té",
    "serie_que_ve": "Westworld",
    "color": "azul",
    "mascota": "caballo"
  },
  {
    "nacionalidad": "ecuatoriano",
    "bebida_que_toma": "leche",
    "serie_que_ve": "El Patrón del Mal",
    "color": "roja",
    "mascota": "pez"
  },
  {
    "nacionalidad": "venezolano",
    "bebida_que_toma": "cerveza",
    "serie_que_ve": "Breaking Bad",
    "color": "verde",
    "mascota": "perro"
  },
  {
    "nacionalidad": "brasilero",
    "bebida_que_toma": "café",
    "serie_que_ve": "The Boys",
    "color": "blanca",
    "mascota": "pájaro"
  }
]

pd.DataFrame(posible_solucion_chatgpt_2)

,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,peruana,agua,Dexter,amarilla,gato
1,colombiana,té,Westworld,azul,caballo
2,ecuatoriano,leche,El Patrón del Mal,roja,pez
3,venezolano,cerveza,Breaking Bad,verde,perro
4,brasilero,café,The Boys,blanca,pájaro


In [31]:
verificar_condiciones(posible_solucion_chatgpt_2)


Solución INVÁLIDA: Número total de fallos: 5
Las siguientes no se cumplen:

1. El brasilero vive en la casa roja
3. El ecuatoriano toma té
7. El dueño de la casa verde bebe café
11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato
14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua

Por favor intenta nuevamente.


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,False
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,False
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",True
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,False
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",True


### Claude 3.5

In [5]:
# Claude 3.5
posible_solucion_claude_35_1 = [
  {
    "nacionalidad": "peruana",
    "bebida_que_toma": "agua",
    "serie_que_ve": "El Patrón del Mal",
    "color": "amarilla",
    "mascota": "gato"
  },
  {
    "nacionalidad": "ecuatoriana",
    "bebida_que_toma": "té",
    "serie_que_ve": "Dexter",
    "color": "azul",
    "mascota": "caballo"
  },
  {
    "nacionalidad": "brasilera",
    "bebida_que_toma": "leche",
    "serie_que_ve": "The Boys",
    "color": "roja",
    "mascota": "pájaros"
  },
  {
    "nacionalidad": "venezolana",
    "bebida_que_toma": "café",
    "serie_que_ve": "Breaking Bad",
    "color": "verde",
    "mascota": "perro"
  },
  {
    "nacionalidad": "colombiana",
    "bebida_que_toma": "cerveza",
    "serie_que_ve": "Westworld",
    "color": "blanca",
    "mascota": "pez"
  }
]

pd.DataFrame(posible_solucion_claude_35_1) 


,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,peruana,agua,El Patrón del Mal,amarilla,gato
1,ecuatoriana,té,Dexter,azul,caballo
2,brasilera,leche,The Boys,roja,pájaros
3,venezolana,café,Breaking Bad,verde,perro
4,colombiana,cerveza,Westworld,blanca,pez


In [6]:
verificar_condiciones(posible_solucion_claude_35_1)


Solución INVÁLIDA: Número total de fallos: 4
Las siguientes no se cumplen
9. El dueño de la casa amarilla ve "Dexter"
11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato
13. El propietario que ve "Breaking Bad" toma cerveza
14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua

Por favor intenta nuevamente.


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,True
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,True
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",True
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,True
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",False


In [10]:
# Claude posible solución v.3. 

posible_solucion_claude_35_v3 = [
  {
    "nacionalidad": "peruana",
    "bebida_que_toma": "agua",
    "serie_que_ve": "El Patrón del Mal",
    "color": "roja",
    "mascota": "pez"
  },
  {
    "nacionalidad": "ecuatoriana",
    "bebida_que_toma": "té",
    "serie_que_ve": "The Boys",
    "color": "azul",
    "mascota": "pájaros"
  },
  {
    "nacionalidad": "brasilera",
    "bebida_que_toma": "leche",
    "serie_que_ve": "Dexter",
    "color": "amarilla",
    "mascota": "caballo"
  },
  {
    "nacionalidad": "colombiana",
    "bebida_que_toma": "café",
    "serie_que_ve": "Westworld",
    "color": "verde",
    "mascota": "gato"
  },
  {
    "nacionalidad": "venezolana",
    "bebida_que_toma": "cerveza",
    "serie_que_ve": "Breaking Bad",
    "color": "blanca",
    "mascota": "perro"
  }
]

pd.DataFrame(posible_solucion_claude_35_v3)

,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,peruana,agua,El Patrón del Mal,roja,pez
1,ecuatoriana,té,The Boys,azul,pájaros
2,brasilera,leche,Dexter,amarilla,caballo
3,colombiana,café,Westworld,verde,gato
4,venezolana,cerveza,Breaking Bad,blanca,perro


In [11]:
verificar_condiciones(posible_solucion_claude_35_v3)


Solución INVÁLIDA: Número total de fallos: 3
Las siguientes no se cumplen
1. El brasilero vive en la casa roja
11. El vecino que ve "El Patrón del Mal" vive al lado del que tiene un gato
14. El vecino que ve "El Patrón del Mal" vive al lado del que toma agua

Por favor intenta nuevamente.


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,False
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,True
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",True
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,True
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",True


In [12]:
posible_solucion_claude_35_v5 = [
  {
    "nacionalidad": "peruana",
    "bebida_que_toma": "agua",
    "serie_que_ve": "Breaking Bad",
    "color": "amarilla",
    "mascota": "caballo"
  },
  {
    "nacionalidad": "colombiana",
    "bebida_que_toma": "cerveza",
    "serie_que_ve": "El Patrón del Mal",
    "color": "azul",
    "mascota": "pez"
  },
  {
    "nacionalidad": "brasilera",
    "bebida_que_toma": "leche",
    "serie_que_ve": "Dexter",
    "color": "roja",
    "mascota": "gato"
  },
  {
    "nacionalidad": "ecuatoriana",
    "bebida_que_toma": "té",
    "serie_que_ve": "The Boys",
    "color": "verde",
    "mascota": "pájaros"
  },
  {
    "nacionalidad": "venezolana",
    "bebida_que_toma": "café",
    "serie_que_ve": "Westworld",
    "color": "blanca",
    "mascota": "perro"
  }
]

pd.DataFrame(posible_solucion_claude_35_v5) 


,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,peruana,agua,Breaking Bad,amarilla,caballo
1,colombiana,cerveza,El Patrón del Mal,azul,pez
2,brasilera,leche,Dexter,roja,gato
3,ecuatoriana,té,The Boys,verde,pájaros
4,venezolana,café,Westworld,blanca,perro


In [13]:
verificar_condiciones(posible_solucion_claude_35_v5)


Solución INVÁLIDA: Número total de fallos: 5
Las siguientes no se cumplen
5. La colombiana ve "Westworld"
7. El dueño de la casa verde bebe café
9. El dueño de la casa amarilla ve "Dexter"
12. El hombre que tiene un caballo vive al lado del que ve "Dexter"
13. El propietario que ve "Breaking Bad" toma cerveza

Por favor intenta nuevamente.


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,True
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,True
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",False
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,False
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",False


### Gemini 2.5 Flash

In [16]:
posible_solucion_gemini_25_flash_v1 = [
{
"nacionalidad": "Peruana",
"bebida_que_toma": "Agua",
"serie_que_ve": "Dexter",
"color": "Amarilla",
"mascota": "Gato"
},
{
"nacionalidad": "Ecuatoriana",
"bebida_que_toma": "Té",
"serie_que_ve": "El Patrón del Mal",
"color": "Azul",
"mascota": "Caballo"
},
{
"nacionalidad": "Brasileña",
"bebida_que_toma": "Leche",
"serie_que_ve": "The Boys",
"color": "Roja",
"mascota": "Pájaros"
},
{
"nacionalidad": "Colombiana",
"bebida_que_toma": "Café",
"serie_que_ve": "Westworld",
"color": "Verde",
"mascota": "Pez"
},
{
"nacionalidad": "Venezolana",
"bebida_que_toma": "Cerveza",
"serie_que_ve": "Breaking Bad",
"color": "Blanca",
"mascota": "Perro"
}
]

pd.DataFrame(posible_solucion_gemini_25_flash_v1)


,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,Peruana,Agua,Dexter,Amarilla,Gato
1,Ecuatoriana,Té,El Patrón del Mal,Azul,Caballo
2,Brasileña,Leche,The Boys,Roja,Pájaros
3,Colombiana,Café,Westworld,Verde,Pez
4,Venezolana,Cerveza,Breaking Bad,Blanca,Perro


In [17]:
verificar_condiciones(posible_solucion_gemini_25_flash_v1)


Solución VÁLIDA!!!


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,True
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,True
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",True
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,True
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",True


### Gemini 2.5 pro

In [14]:
posible_solucion_gemini_25_pro_v1 = [
  {
    "nacionalidad": "Peruana",
    "bebida_que_toma": "Agua",
    "serie_que_ve": "Dexter",
    "color": "Amarilla",
    "mascota": "Gato"
  },
  {
    "nacionalidad": "Ecuatoriano",
    "bebida_que_toma": "Té",
    "serie_que_ve": "El Patrón del Mal",
    "color": "Azul",
    "mascota": "Caballo"
  },
  {
    "nacionalidad": "Brasilero",
    "bebida_que_toma": "Leche",
    "serie_que_ve": "The Boys",
    "color": "Roja",
    "mascota": "Pájaros"
  },
  {
    "nacionalidad": "Colombiana",
    "bebida_que_toma": "Café",
    "serie_que_ve": "Westworld",
    "color": "Verde",
    "mascota": "Peces"
  },
  {
    "nacionalidad": "Venezolano",
    "bebida_que_toma": "Cerveza",
    "serie_que_ve": "Breaking Bad",
    "color": "Blanca",
    "mascota": "Perro"
  }
]

pd.DataFrame(posible_solucion_gemini_25_pro_v1) 


,nacionalidad,bebida_que_toma,serie_que_ve,color,mascota
0,Peruana,Agua,Dexter,Amarilla,Gato
1,Ecuatoriano,Té,El Patrón del Mal,Azul,Caballo
2,Brasilero,Leche,The Boys,Roja,Pájaros
3,Colombiana,Café,Westworld,Verde,Peces
4,Venezolano,Cerveza,Breaking Bad,Blanca,Perro


In [15]:
verificar_condiciones(posible_solucion_gemini_25_pro_v1)


Solución VÁLIDA!!!


,indice,en espanhol,se cumple
0,0,En una calle hay cinco casas,True
1,1,El brasilero vive en la casa roja,True
2,2,El venezolano tiene un perro como mascota,True
3,3,El ecuatoriano toma té,True
4,4,La peruana vive en la primera casa,True
5,5,"La colombiana ve ""Westworld""",True
6,6,La casa verde está inmediatamente a la izquier...,True
7,7,El dueño de la casa verde bebe café,True
8,8,"El propietario que ve ""The Boys"" cría pájaros",True
9,9,"El dueño de la casa amarilla ve ""Dexter""",True


In [ ]:
posible_solicion_gemini_25_flash_v1